# Ejercicio 7 historico indices Yahoo

- Obtén los datos históricos de cada índice (60 días)
- Recomendación: Yahoo Finance. Tiempo objetivo: 45 minutos

In [2]:
!pip install rcurl

You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup #https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe#/
import datetime
import time
from tqdm import tqdm
import math
from rcurl import get_curl #https://pypi.org/project/rcurl/
from io import BytesIO
from time import mktime
from datetime import timedelta
from datetime import datetime

Recupero los datos en bruto de Yahoo finance, realizando el chequeo del crumb.

In [4]:
def _get_crumbs_and_cookies(stock):
    """
    get crumb and cookies for historical data csv download from yahoo finance
    parameters: stock - short-handle identifier of the company 
    returns a tuple of header, crumb and cookie
    """
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    
    with requests.session():
        
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

Establecemos el rango de fechas que queremos importar.

Yahoo utiliza formato de fecha poxis - time unix
(cantidad de segundos transcurridos desde la medianoche UTC del 1 de enero de 1970, teniendo cada día 86400 segundos)

Convertimos la fecha a formato poxis

In [5]:
def convert_to_unix(date):
    """
    converts date to unix timestamp    
    parameters: date - in format (yyyy-mm-dd)    
    returns integer unix timestamp
    """
    datum = datetime.strptime(date, '%Y-%m-%d')
    
    return int(mktime(datum.timetuple()))

In [7]:
def load_csv_data(stock, interval='1d', day_begin='20-03-2018', day_end='20-06-2018'):
    """
    queries yahoo finance api to receive historical data in csv file format
    
    parameters: 
        stock - short-handle identifier of the company
        
        interval - 1d, 1wk, 1mo - daily, weekly monthly data
        
        day_begin - starting date for the historical data (format: dd-mm-yyyy)
        
        day_end - final date of the data (format: dd-mm-yyyy)
    
    returns a list of comma seperated value lines
    """
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)       
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/'               '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}'               .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')

In [8]:
tickers_de_indices=("%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", 
                    "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI")

Definimos las fechas:

In [14]:
ventana=60

hoy = datetime.now()
fecha_fin = str(hoy.now())
fecha_fin = fecha_fin[0:10]
print(fecha_fin)

fecha_inicial = hoy-timedelta(ventana)
fecha_inicial = str(fecha_inicial)
fecha_inicial = fecha_inicial[0:10]
print(fecha_inicial)

2020-06-30
2020-05-01


Descargamos los datos

In [15]:
for benchmark in tickers_de_indices:
    
    print("descargando "+benchmark)
    
    try:
        DF=load_csv_data(benchmark, interval='1d', day_begin=fecha_inicial, day_end=fecha_fin)
    except:
        print("No nos hemos podido descargar el indice")
    
    DF=pd.DataFrame(DF)
    DF = DF.iloc[:,0].str.split(",", n = 7, expand = True)
    DF.columns=DF.iloc[0,:]
    DF=DF.iloc[1:DF.shape[0],:]

descargando %5EBFX
descargando %5EBVSP
descargando %5EDJI
descargando %5EFCHI
descargando %5EFTSE
descargando %5EGDAXI
descargando %5EHSI
descargando %5EIBEX
descargando %5EMXX
descargando %5EJKSE
descargando %5EMERV
descargando %5EOMXSPI
descargando %5EOSEAX
descargando %5ESSMI
descargando %5ESTI


In [17]:
DF.head()

,Date,Open,High,Low,Close,Adj Close,Volume
1,2020-05-04,2555.669922,2575.139893,2536.629883,2563.689941,2563.689941,349485700
2,2020-05-05,2579.100098,2595.129883,2572.360107,2572.360107,2572.360107,220314200
3,2020-05-06,2566.689941,2608.010010,2565.939941,2591.600098,2591.600098,294626700
4,2020-05-08,2600.379883,2602.429932,2579.360107,2591.879883,2591.879883,330465700
5,2020-05-11,2597.850098,2611.739990,2590.659912,2611.310059,2611.310059,260504400


## Otra manera de hacerlo

In [1]:
import urllib.request
import requests
from io import StringIO
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
import time

In [ ]:
tickers_de_indices=["%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"]
ventana=60

In [ ]:
def datos_indices (tickers_de_indices,ventana):
    for benchmark in tickers_de_indices:
        print(benchmark+" descargando")
        
        # Obtengo las cookies de su web 
        cookieProcessor = urllib.request.HTTPCookieProcessor()
        opener = urllib.request.build_opener(cookieProcessor)
        
        # Una vez obtenida la cookie hago el res la url y obtengo todo el html para calcular crumb
        res=str(opener.open("https://es.finance.yahoo.com/quote/"+benchmark+"?ltr=1").read())
        
        # Buscamos y filtramos CrumbStore obteniendo el "pass" del crumb
        posicion=res.find('CrumbStore')
        crumb=res[posicion:posicion+35].split()
        inicio=crumb[0].find(':"')
        final=crumb[0].find('"}')
        crumb=crumb[0][inicio+2:final]
        
        # Convertimos las fechas al formato correspondiente y calculamos con la ventana el rango a obteer
        fecha_fin=int(round(time.time()))
        fecha_inicio=datetime.now()-timedelta(ventana)
        fecha_inicio=int(round(fecha_inicio.timestamp()))
        
        # Construimos la url con todo lo calculado anteriormente y guardamos los .csv en DF
        url="https://query1.finance.yahoo.com/v7/finance/download/"+benchmark+"?period1="+str(fecha_inicio)+"&period2="+str(fecha_fin)+"&interval=1d&events=history&crumb="+str(crumb)
        req = requests.get(url)
        data = StringIO(req.text)
        indices=pd.read_csv(data)
        
        #Guardamos los nombres de los df con el nombre del inice sin el %5 pasado en tickers_de_indices 
        globals()[benchmark.replace("%5", "").replace(".","_")]=indices
        print("Guardamos datos: " +benchmark+ " con nombre: " +benchmark.replace("%5", "").replace(".","_"))
        print("")

In [ ]:
datos_indices(tickers_de_indices,ventana)

In [ ]:
EJKSE

## Otra manera de hacerlo

In [ ]:
import re
import io
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
tickers_de_indices = [
    "%5EBFX", "%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE",
    "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE",
    "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"
]

In [ ]:
def obtener_indice(url, cookies):
    try:
        benchmark = requests.get(url, cookies=cookies).text
        benchmark = io.StringIO(benchmark)
        
    except Exception as e:
        
        print("No nos hemos podido descargar el indice")
        print("Este es el mensaje de error que ha dado:")
        print(e)

        return("Sin datos")

    return benchmark

In [ ]:
ventana = 60

In [ ]:
for benchmark in tickers_de_indices:

    print(f'{benchmark} descargando')

    # html de la web del benchmark
    request = requests.get(f"https://es.finance.yahoo.com/quote/{benchmark}?ltr=1")
    res = request.text

    # Obtenemos el CrumbStore
    pattern_crumbstore = r'\"CrumbStore\"\:{\"crumb\":\"(?P<crumb>.*?)\"}'
    crumb = re.search(pattern_crumbstore, res).groupdict().get('crumb')
    crumb = crumb.replace("\n", "")  # Algunas veces, el crumb tiene una función de escape (un salto de página).

    # Fechas inicio y fin
    fecha_inicial = (datetime.now() - relativedelta(days=ventana)).strftime('%s')
    fecha_fin = datetime.now().strftime('%s')

    # Construimos la url para bajarnos los datos
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{benchmark}?period1={fecha_inicial}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}"

    # Obtenemos indice a partir de la url
    benchmark_io = obtener_indice(url, request.cookies)
    data = pd.read_csv(benchmark_io)

    globals()[benchmark] = data

    print(data)

## Otra manera de hacerlo

In [ ]:
import json
import requests
from rcurl import get_curl
import time
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
ventana = 60
fecha_fin=int(time.time())
fecha_inicio=int(fecha_fin-86400*ventana)

In [ ]:
indices = ["%5EBFX","%5EBVSP", "%5EDJI", "%5EFCHI", "%5EFTSE", "%5EGDAXI", "%5EHSI", "%5EIBEX", "%5EMXX", "%5EJKSE", "%5EMERV", "%5EOMXSPI", "%5EOSEAX", "%5ESSMI", "%5ESTI"]

for benchmark in indices:
    url = f"https://es.finance.yahoo.com/quote/{benchmark}?ltr=1"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html')
    linea=str(soup)
    linea=linea[linea.find('CrumbStore'):(linea.find('CrumbStore')+50)]
    crumb=linea[(linea.find(':"')+2):linea.find('"}')]
    new_url=f'https://query1.finance.yahoo.com/v7/finance/download/{benchmark}?period1={fecha_inicio}&period2={fecha_fin}&interval=1d&events=history&crumb={crumb}'
    contenido=requests.get(new_url)
    tabla = pd.read_csv(new_url, encoding='utf-8')
    print(f'Descargando datos del indice {benchmark}')
    print(tabla)

## Otra manera de hacerlo

In [ ]:
from datetime import datetime, timedelta
from time import mktime
import requests    
from bs4 import BeautifulSoup
import re   
import pandas as pd
from tqdm import tqdm

In [ ]:
indices = ["%5EBFX","%5EBVSP", "%5EDJI",
           "%5EFCHI", "%5EFTSE", "%5EGDAXI",
           "%5EHSI", "%5EIBEX", "%5EMXX",
           "%5EJKSE", "%5EMERV", "%5EOMXSPI",
           "%5EOSEAX", "%5ESSMI", "%5ESTI"]

ventana = 60
end_date = datetime.today()
start_date = end_date.today() - timedelta(days=ventana)

end_date_unix = int(mktime(end_date.date().timetuple()))
start_date_unix = int(mktime(start_date.date().timetuple()))

In [ ]:
def get_index(index:str, start_date_unix:int,  end_date_unix:int):
    
    result = None
    try:
        url = f"https://es.finance.yahoo.com/quote/{index}?ltr=1"
        website = requests.get(url) 
        soup = BeautifulSoup(website.text)
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))[0]
        url2 = f"https://query1.finance.yahoo.com/v7/finance/download/{index}?period1={start_date_unix}&period2={end_date_unix}&interval=1d&events=history&crumb={crumb}"
        website = requests.get(url2).text.split('\n')
        table = [line.split(',') for line in website]
        result = pd.DataFrame(data=table[1:], columns=table[0])
        result.set_index('Date', inplace=True)
        result['Open'] = result['Open'].astype(float)
        result['High'] = result['High'].astype(float)
        result['Low'] = result['Low'].astype(float)
        result['Close'] = result['Close'].astype(float)
        result['Adj Close'] = result['Adj Close'].astype(float)
        result['Volume'] = result['Volume'].astype(float)
    
    except Exception as e:
        print(f'Index {index} could not be downloaded.')
        print(e)

    return result

In [ ]:
result = {index:get_index(index, start_date_unix, end_date_unix) for index in tqdm(indices)}

In [ ]:
result[indices[0]].head()